## Corpora

In [1]:
from ipynb.fs.full.koselleck import *
# pd.options.display.latex.repr=True

In [2]:
get_nbr_simple_sattelzeit('culture,liberal,liberty')

,Neighborhood 1 (Pre-Sattelzeit),Neighborhood 2 (Post-Sattelzeit),Change Rank,Change Percentile
word,,,,
liberal,"<u>-polite</u> (-10), <u>-affable</u> (-10), <u>-endowments</u> (-10), <u>-extensive</u> (-10), <u>-education</u> (-10), <u>-learning</u> (-10), <u>-benevolent</u> (-10), <u>-refined</u> (-10), <u>-talents</u> (-10), <u>-munificence</u> (-8), <u>-accomplishments</u> (-8), <u>-mechanic</u> (-8), <u>-frugal</u> (-8), <u>-science</u> (-8), <u>-courteous</u> (-7), <u>-sciences</u> (-7), <u>-benevolence</u> (-7), <u>-assiduous</u> (-6), <u>-amiable</u> (-6), <u>-probity</u> (-6), <u>-genteel</u> (-5), humane (-4), <u>-attainments</u> (-4), <u>-beneficent</u> (-4), <u>-industry</u> (-4), <u>-arts</u> (-3), <u>-industrious</u> (-3), <u>-commendable</u> (-3), munificent (-2), <u>-hospitality</u> (-2), <u>-studious</u> (-2), <u>-laborious</u> (-2), <u>-consummate</u> (-2), <u>-generosity</u> (-2), <u>-proficient</u> (-1), <u>-indigent</u> (-1), <u>-profession</u> (-1), <u>-literature</u> (-1), <u>-beneficence</u> (-1), <u>-manly</u> (-1), <u>-erudition</u> (-1), <u>-studied</u> (-1), <u>-temperance</u> (-1), <u>-exemplary</u> (-1), <u>-ingenuity</u> (-1), generous (+0), liberality (+1), charitable (+3), disinterested (+3)","<u>+whig</u> (+10), <u>+radical</u> (+10), <u>+tory</u> (+10), <u>+moderate</u> (+10), <u>+enlightened</u> (+10), <u>+republican</u> (+10), <u>+ministerial</u> (+10), <u>+whigs</u> (+10), <u>+supporters</u> (+10), <u>+patriotic</u> (+10), <u>+zealous</u> (+9), <u>+equitable</u> (+9), <u>+electors</u> (+9), <u>+advocates</u> (+9), <u>+strenuous</u> (+8), <u>+dissenters</u> (+7), <u>+opposition</u> (+7), <u>+politicians</u> (+6), <u>+parliamentary</u> (+6), <u>+reform</u> (+6), <u>+political</u> (+5), <u>+monarchical</u> (+4), <u>+constitutional</u> (+4), <u>+leaders</u> (+4), <u>+electoral</u> (+3), disinterested (+3), <u>+reforming</u> (+3), charitable (+3), <u>+premier</u> (+2), <u>+policy</u> (+2), <u>+unpopular</u> (+2), <u>+government</u> (+2), liberality (+1), <u>+supporter</u> (+1), <u>+evangelical</u> (+1), <u>+judicious</u> (+1), <u>+cordial</u> (+1), <u>+partisans</u> (+1), <u>+reformers</u> (+1), <u>+democracy</u> (+1), <u>+ballot</u> (+1), generous (+0), munificent (-2), humane (-4)",376.0,97.0
culture,"<u>-planting</u> (-10), <u>-soils</u> (-10), <u>-vegetation</u> (-10), <u>-sowing</u> (-10), <u>-tillage</u> (-10), <u>-husbandry</u> (-10), <u>-soil</u> (-10), <u>-plants</u> (-10), <u>-vines</u> (-10), <u>-gardening</u> (-10), <u>-fruit</u> (-10), <u>-improvement</u> (-10), <u>-vegetables</u> (-10), <u>-manure</u> (-10), <u>-plant</u> (-10), <u>-seeds</u> (-8), <u>-flax</u> (-8), <u>-gardeners</u> (-7), <u>-fruits</u> (-7), <u>-shrubs</u> (-7), <u>-herbs</u> (-7), <u>-feeding</u> (-6), <u>-improving</u> (-5), <u>-vineyard</u> (-4), agriculture (-3), <u>-flowering</u> (-3), <u>-managing</u> (-3), <u>-feeds</u> (-2), <u>-roots</u> (-1), cultivation (+0), cultivating (+0), growth (+8)","<u>+intellectual</u> (+10), <u>+education</u> (+10), <u>+cultivated</u> (+10), <u>+attainments</u> (+10), <u>+refinement</u> (+10), <u>+advancement</u> (+10), <u>+training</u> (+10), <u>+capacities</u> (+10), <u>+industry</u> (+10), <u>+knowledge</u> (+10), <u>+learning</u> (+10), <u>+intellect</u> (+9), <u>+science</u> (+9), growth (+8), <u>+refinements</u> (+8), <u>+literature</u> (+8), <u>+cultivate</u> (+7), <u>+morals</u> (+7), <u>+endowments</u> (+6), <u>+activity</u> (+6), <u>+product</u> (+5), <u>+pursuits</u> (+5), <u>+attainment</u> (+5), <u>+proficiency</u> (+3), <u>+philosophy</u> (+3), <u>+organization</u> (+3), <u>+faculties</u> (+2), <u>+fertility</u> (+2), <u>+erudition</u> (+2), <u>+tastes</u> (+2), <u>+acquiring</u> (+2), <u>+morality</u> (+2), <u>+social</u> (+1), <u>+thrift</u> (+1), <u>+intelligence</u> (+1), <u>+accomplishments</u> (+1), <u>+instruction</u> (+1), <u>+opulence</u> (+1), <u>+teaching</u> (+1), cultivation (+0), cultivating (+0), agriculture (-3)",1327.0,88.0
liberty,"<u>-gover

In [11]:
C=None
def get_corpus():
    global C
    if C is None:
        C=lltk.load(CNAME)
    return C

In [12]:
C=get_corpus()

In [13]:
len(C.meta)

50449

In [14]:
C.meta

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,ObjectType,ISSN,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus
id,,,,,,,,,,,,,,,,,,,,,
1010/Advertisement/2032695,,Advertisements,1010,10101,1618364638,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,Advertisement,,ENG,Historical Periodicals,1155,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032695.xml,1010,BPO
1010/Advertisement/2032690,,MAPPIN & WEBB Ltd.,1010,10101,1618367119,2032690,The Sketch,London,Illustrated London News,,...,Advertisement,,ENG,Historical Periodicals,131,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032690.xml,1010,BPO
1010/Unknown/2032695,,THE Old Etonians beat Edinburgh University by five goal four in a football match at Edinburgh on Tuesday,1010,10101,1618367761,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,,,ENG,Historical Periodicals,55,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Unknown/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Unknown/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Unknown/2032695.xml,1010,BPO
1010/News/2032690,,"GIVEN COMING-OF,-AGE FESTIVITIES, JUST LIKE A SON AND HEIR",1010,10101,1618367834,2032690,The Sketch,London,Illustrated London News,,...,News,,ENG,Historical Periodicals,64,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032690.xml,1010,BPO
1010/News/2032695,,Graphic,1010,10101,1618372563,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,News,,ENG,Historical Periodicals,0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032695.xml,1010,BPO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996/Advertisement/2032697,,TURKEY,1996,19960226,1689079340,2032697,London Life,London,,,...,Advertisement,,ENG,Historical Periodicals,233,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Advertisement/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Advertisement/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Advertisement/2032697.xml,1996,BPO
1996/Unknown/2032697,,BELGRAVIA DECLARES METER WAR,1996,19960226,1689080163,2032697,London Life,London,,,...,,,ENG,Historical Periodicals,894,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Unknown/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Unknown/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Unknown/2032697.xml,1996,BPO
1996/Review/2032697,,The Absorbing Fire,1996,19960226,1689080953,2032697,London Life,London,,,...,Review,,ENG,Historical Periodicals,68,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Review/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Review/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Review/2032697.xml,1996,BPO


## OCR

In [15]:
from lltk import get_wordlist

In [16]:
def measure_ocr_accuracy(txt_or_tokens_or_tokend,lang='en'):
    wordlist={w.lower() for w in get_wordlist(lang=lang)}
    if type(txt_or_tokens_or_tokend) in {str}:
        tokend=Counter(tokenize(txt_or_tokens_or_tokend))
    elif type(txt_or_tokens_or_tokend) in {tuple,list}:
        tokens=Counter(txt_or_tokens_or_tokend)
    elif type(txt_or_tokens_or_tokend) in {dict,Counter}:
        tokend=txt_or_tokens_or_tokend
    else: 
        raise Exception("Function `measure_ocr_accuracy(txt_or_tokens_or_tokend)` must take text string or list of tokens or dictionary.")
    numwords=sum(tokend.values())
    numrecog=sum(
        val for tok,val in tokend.items()
        if tok.lower() in wordlist
    )
    return {
        'num_words':numwords,
        'num_recog':numrecog,
        'ocr_accuracy':numrecog/numwords
    }

In [17]:
def writeiter_jsonl(fnfn,iterr):
    import jsonlines
    with jsonlines.open(fnfn,'w') as writer:
        for i,dx in enumerate(iterr):
            writer.write(dx)

In [18]:

def do_measure_ocr_accuracy_corpus(obj):
    idx,path_freqs=obj
    od={'id':idx}
    if not os.path.exists(path_freqs): return od
    with open(path_freqs) as f: d=ujson.load(f)
    odx=measure_ocr_accuracy(d)
    return {**od, **odx}
    
def measure_ocr_accuracy_corpus(self,num_proc=1,**y):
    ofn=os.path.join(self.path_data, 'data.ocr_accuracy.jsonl')
    if os.path.exists(ofn): return pd.read_json(ofn,lines=True).set_index('id')
    objs=[(t.id,t.path_freqs) for t in self.texts()]
    iterr=pmap_iter(
        do_measure_ocr_accuracy_corpus,
        objs,
        num_proc=num_proc,
        **y
    )
    return writeiter_jsonl(ofn, iterr)

In [19]:
dfocr=measure_ocr_accuracy_corpus(C,num_proc=4)
dfocr

,num_words,num_recog,ocr_accuracy
id,,,
1010/Advertisement/2032695,48885.0,42828.0,0.876097
1010/Advertisement/2032690,42862.0,36991.0,0.863026
1010/Unknown/2032695,65408.0,63042.0,0.963827
1010/News/2032690,9280.0,8580.0,0.924569
1010/News/2032695,3771.0,3488.0,0.924954
...,...,...,...
1996/Advertisement/2032697,718.0,639.0,0.889972
1996/Unknown/2032697,13975.0,13174.0,0.942683
1996/Review/2032697,2868.0,2671.0,0.931311


In [20]:
# stop

In [21]:
# !cat /home/ryan/lltk_data/corpora/bpo/data/data.ocr_accuracy.jsonl

In [22]:
# measure_ocr_accuracy(d)

In [23]:
# stop

In [24]:
dfmeta_orig=read_df('/home/ryan/lltk_data/corpora/bpo/metadata-full.csv')
dfmeta_orig

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words
0,19061020,7696950,Advertisement,2563,Musical standard,London,Reeves and Turner,NaN,[iii],[iii],Advertisement,2043-4847,ENG,Historical Periodicals,1209
1,18971201,8099455,FINGERING THE VIOLA.,2896,The Violin times : a monthly journal for professional and amateur violinists and quartet players,London,W. Reeves,NaN,32,32,General Information,2044-6608,ENG,Historical Periodicals,494
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762
3,19500322,1618790372,"Macmillan & Co., Ltd.",2032697,The Tatler and Bystander,NaN,NaN,NaN,562,NaN,Advertisement,NaN,ENG,Historical Periodicals,95
4,18990601,7783559,Sweet Vales of Devonia.,2567,"Musical times and singing class circular, 1844-1903",London / New York,Novello,NaN,1,4,Poem,NaN,ENG,Historical Periodicals,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869239,19020322,9324438,Advertisement,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,383,383,Advertisement,0950-9852,ENG,Historical Periodicals,2381
2869240,19580924,1618649620,PETER SCOTT & CO LTD,2032697,The Tatler and Bystander,NaN,NaN,NaN,619,NaN,Advertisement,NaN,ENG,Historical Periodicals,33
2869241,19180921,1618735471,MELLIN'S FOOD WORKS,2032689,The Sphere,London,Illustrated London News,NaN,i,NaN,Advertisement,NaN,ENG,Historical Periodicals,65
2869242,18970929,1638088199,SMALL TALK,2032690,The Sketch,London,Illustrated London News,NaN,389,NaN,NaN,NaN,ENG,Historical Periodicals,6776


In [25]:
dfmeta_orig['year']=dfmeta_orig['NumericPubDate'].apply(lambda x: int(str(x)[:4]))
dfmeta_orig['period']=dfmeta_orig.year.apply(lambda y: periodize_bystep(y,25,1600))
dfmeta_orig

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,year,period
0,19061020,7696950,Advertisement,2563,Musical standard,London,Reeves and Turner,NaN,[iii],[iii],Advertisement,2043-4847,ENG,Historical Periodicals,1209,1906,1900-1925
1,18971201,8099455,FINGERING THE VIOLA.,2896,The Violin times : a monthly journal for professional and amateur violinists and quartet players,London,W. Reeves,NaN,32,32,General Information,2044-6608,ENG,Historical Periodicals,494,1897,1875-1900
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762,1880,1875-1900
3,19500322,1618790372,"Macmillan & Co., Ltd.",2032697,The Tatler and Bystander,NaN,NaN,NaN,562,NaN,Advertisement,NaN,ENG,Historical Periodicals,95,1950,1950-1975
4,18990601,7783559,Sweet Vales of Devonia.,2567,"Musical times and singing class circular, 1844-1903",London / New York,Novello,NaN,1,4,Poem,NaN,ENG,Historical Periodicals,35,1899,1875-1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869239,19020322,9324438,Advertisement,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,383,383,Advertisement,0950-9852,ENG,Historical Periodicals,2381,1902,1900-1925
2869240,19580924,1618649620,PETER SCOTT & CO LTD,2032697,The Tatler and Bystander,NaN,NaN,NaN,619,NaN,Advertisement,NaN,ENG,Historical Periodicals,33,1958,1950-1975
2869241,19180921,1618735471,MELLIN'S FOOD WORKS,2032689,The Sphere,London,Illustrated London News,NaN,i,NaN,Advertisement,NaN,ENG,Historical Periodicals,65,1918,1900-1925
2869242,18970929,1638088199,SMALL TALK,2032690,The Sketch,London,Illustrated London News,NaN,389,NaN,NaN,NaN,ENG,Historical Periodicals,6776,1897,1875-1900


In [26]:
# orig_stats = pd.DataFrame(
#     dict(
#         period=i,
#         num_journals=gdf.Publication_PublicationID.nunique(),
#         num_articles=gdf.RecordID.nunique()
#     ) for i,gdf in dfmeta_orig.groupby('period')
# ).set_index('period')
# orig_stats

In [27]:
idf=dfmeta_orig.query('1670<=year<2000')
idf['num_journals']=idf['Publication_PublicationID']
idf['num_articles']=idf['RecordID']
idf['period']=idf.year.apply(lambda y: periodize(y,25))
orig_stats=idf[['period','num_journals','num_articles']].groupby('period').nunique()#.iloc[1:]
orig_stats

,num_journals,num_articles
period,,
1675-1700,21,3918
1700-1725,44,13661
1725-1750,38,42427
1750-1775,45,70245
1775-1800,47,145531
1800-1825,89,202538
1825-1850,123,279701
1850-1875,122,445557
1875-1900,141,699344


In [28]:
# C.meta.query('1710<year<1720').Publication_Title.value_counts()
meta=C.meta.query('1600<=year<2100')

In [29]:
s=meta.Publication_Title.value_counts()
# s

In [30]:
dfpub2=meta.groupby('Publication_Title').agg({'year':['min','median','max']})
dfpub2.columns=['_'.join(x) for x in dfpub2.columns.to_flat_index()]
dfpub2['size']=s
dfpub2=dfpub2.sort_values('size',ascending=False)
dfpub2

,year_min,year_median,year_max,size
Publication_Title,,,,
"The Gentleman's Magazine: and historical chronicle, Jan. 1736-Dec. 1833",1736,1784.0,1833,811
The Athenaeum,1830,1876.0,1921,724
Illustrated Sporting and Dramatic News,1642,1905.0,1943,672
Examiner,1808,1845.0,1881,595
The Sketch,1864,1925.0,1997,584
...,...,...,...,...
"General history of discoveries and improvements : in useful arts, particularly in the great branches of commerce, navigation, and plantation, in all parts of the known world",1726,1726.0,1726,1
Fortnightly,1934,1934.0,1934,1
"The History of learning, 1694",1694,1694.0,1694,1


In [31]:
dfpub2.query('year_min<1700').sort_values('year_min')

,year_min,year_median,year_max,size
Publication_Title,,,,
Illustrated Sporting and Dramatic News,1642,1905.0,1943,672
Heraclitus ridens,1681,1703.0,1718,15
"Democritus ridens, or, Comus and Momus",1681,1681.0,1681,3
"The Observator in question and answer, Apr. 1681- July 1681",1681,1681.0,1681,6
The Observator in dialogue,1682,1685.0,1687,20
"Weekly memorials for the ingenious : or, An account of books lately set forth in several languages",1682,1682.0,1683,10
"Weekly lampoon; or, Satyrical reflections on the last weeks publick news letters and observator, Oct. 1690",1690,1690.0,1690,3
"The History of learning, or, An abstract of several books lately published, as well abroad, as at home / By one of the two authors of the Universal and historical bibliotheque",1691,1691.0,1691,2
Athenian gazette,1691,1693.0,1697,36


## Corp info

In [32]:
# C.meta.path_freqs

In [33]:
C.meta[C.meta.Publication_Title.str.contains('Spectator')]

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,ObjectType,ISSN,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus
id,,,,,,,,,,,,,,,,,,,,,
1711/Unknown/18038,,Untitled item,1711,17110303,6319128,18038,The Spectator (1711),London,Sam. Buckley.,,...,,1358-6149,ENG,Historical Periodicals,1244,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Unknown/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Unknown/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Unknown/18038.xml,1711,BPO
1711/Back_Matter/18038,,Back matter,1711,17110303,6319159,18038,The Spectator (1711),London,Sam. Buckley.,,...,Back Matter,1358-6149,ENG,Historical Periodicals,29,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Back_Matter/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Back_Matter/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Back_Matter/18038.xml,1711,BPO
1711/Review/18038,,Book review,1711,17110305,6313768,18038,The Spectator (1711),London,Sam. Buckley.,,...,Review,1358-6149,ENG,Historical Periodicals,1704,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Review/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Review/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Review/18038.xml,1711,BPO
1711/Advertisement/18038,,Advertisement,1711,17110306,6317135,18038,The Spectator (1711),London,Sam. Buckley.,,...,Advertisement,1358-6149,ENG,Historical Periodicals,833,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Advertisement/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Advertisement/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Advertisement/18038.xml,1711,BPO
1711/Correspondence/18038,,Letter,1711,17110309,6399178,18038,The Spectator (1711),London,Sam. Buckley.,,...,Correspondence,1358-6149,ENG,Historical Periodicals,1187,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Correspondence/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Correspondence/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Correspondence/18038.xml,1711,BPO
1711/General_Information/18038,,ERRATA.,1711,17110416,6314035,18038,The Spectator (1711),London,Sam. Buckley.,,...,General Information,1358-6149,ENG,Historical Periodicals,32,/home/ryan/lltk_data/corpora/bpo/freqs/1711/General_Information/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/General_Information/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/General_Information/18038.xml,1711,BPO
1712/Correspondence/18038,,Letter,1712,17120101,6322677,18038,The Spectator (1711),London,Sam. Buckley.,,...,Correspondence,1358-6149,ENG,Historical Periodicals,1652,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Correspondence/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Correspondence/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Correspondence/18038.xml,1712,BPO
1712/Back_Matter/18038,,Back matter,1712,17120101,6413487,18038,The Spectator (1711),London,Sam. Buckley.,,...,Back Matter,1358-6149,ENG,Historical Periodicals,39,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Back_Matter/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Back_Matter/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Back_Matter/18038.xml,1712,BPO
1712/Advertisement/18038,,Advertisement,1712,17120102,6403376,18038,The Spectator (1711),London,Sam. Buckley.,,...,Advertisement,1358-6149,ENG,Historical Periodicals,319,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Advertisement/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Advertisement/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Advertisement/18038.xml,1712,BPO


In [34]:
# import ujson

# def gettextlen(path):
#     try:
#         with open(path) as f:
#             d=ujson.load(f)
#             return sum(d.values()) if d else np.nan
#     except FileNotFoundError:
#         return np.nan
    
# text_lens = pmap(gettextlen, C.meta.path_freqs)

In [35]:
dfmeta=C.meta
dfmeta['Genre']=dfmeta.ObjectType.replace({'':'Undefined'})
dfmeta

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,ISSN,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus,Genre
id,,,,,,,,,,,,,,,,,,,,,
1010/Advertisement/2032695,,Advertisements,1010,10101,1618364638,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,,ENG,Historical Periodicals,1155,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032695.xml,1010,BPO,Advertisement
1010/Advertisement/2032690,,MAPPIN & WEBB Ltd.,1010,10101,1618367119,2032690,The Sketch,London,Illustrated London News,,...,,ENG,Historical Periodicals,131,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032690.xml,1010,BPO,Advertisement
1010/Unknown/2032695,,THE Old Etonians beat Edinburgh University by five goal four in a football match at Edinburgh on Tuesday,1010,10101,1618367761,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,,ENG,Historical Periodicals,55,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Unknown/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Unknown/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Unknown/2032695.xml,1010,BPO,Undefined
1010/News/2032690,,"GIVEN COMING-OF,-AGE FESTIVITIES, JUST LIKE A SON AND HEIR",1010,10101,1618367834,2032690,The Sketch,London,Illustrated London News,,...,,ENG,Historical Periodicals,64,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032690.xml,1010,BPO,News
1010/News/2032695,,Graphic,1010,10101,1618372563,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,,ENG,Historical Periodicals,0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032695.xml,1010,BPO,News
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996/Advertisement/2032697,,TURKEY,1996,19960226,1689079340,2032697,London Life,London,,,...,,ENG,Historical Periodicals,233,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Advertisement/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Advertisement/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Advertisement/2032697.xml,1996,BPO,Advertisement
1996/Unknown/2032697,,BELGRAVIA DECLARES METER WAR,1996,19960226,1689080163,2032697,London Life,London,,,...,,ENG,Historical Periodicals,894,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Unknown/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Unknown/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Unknown/2032697.xml,1996,BPO,Undefined
1996/Review/2032697,,The Absorbing Fire,1996,19960226,1689080953,2032697,London Life,London,,,...,,ENG,Historical Periodicals,68,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Review/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Review/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Review/2032697.xml,1996,BPO,Review


In [36]:
ybin=25
dfmeta=dfmeta.query('1675<=year<1975')
dfmeta['period']=dfmeta.year.apply(lambda y: periodize(y,ybin))
dfocr

,num_words,num_recog,ocr_accuracy
id,,,
1010/Advertisement/2032695,48885.0,42828.0,0.876097
1010/Advertisement/2032690,42862.0,36991.0,0.863026
1010/Unknown/2032695,65408.0,63042.0,0.963827
1010/News/2032690,9280.0,8580.0,0.924569
1010/News/2032695,3771.0,3488.0,0.924954
...,...,...,...
1996/Advertisement/2032697,718.0,639.0,0.889972
1996/Unknown/2032697,13975.0,13174.0,0.942683
1996/Review/2032697,2868.0,2671.0,0.931311


In [37]:
dfmeta['count']=1
dfinfo1=dfmeta.groupby('period').sum().drop('year',1)
dfinfo1

,count
period,
1675-1700,174
1700-1725,595
1725-1750,1140
1750-1775,1762
1775-1800,2810
1800-1825,5102
1825-1850,6616
1850-1875,9311
1875-1900,12966


## Skip info

In [38]:
# get_skipgrams??

In [39]:
dfskip=get_skipgrams(calc_numlines=True)
dfskip=dfskip.query('1600<year<2000')
dfskip['period']=dfskip.year.apply(lambda y: periodize(y,ybin))

100%|██████████| 289/289 [00:58<00:00,  4.95it/s]


In [40]:
# dfinfo = dfmeta.merge(dfskip,on='period',suffixes=['_meta','']).groupby('period').sum()
# dfinfo
# # dfinfo_str=dfinfo.applymap(lambda x: f'{int(x):,}').rename_axis('Period (20yr)')
# # dfinfo_str.columns=['# Texts','# Sentences','# Words']
# # printm(dfinfo_str.to_markdown())

In [41]:
dfmeta.groupby('period').nunique()

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus,Genre,count
period,,,,,,,,,,,,,,,,,,,,,
1675-1700,1,70,17,92,174,21,23,1,16,1,...,1,1,146,174,174,174,17,1,9,1
1700-1725,1,241,25,327,595,44,47,3,35,1,...,1,2,431,595,595,595,25,1,10,1
1725-1750,1,578,25,422,1140,38,50,4,30,1,...,1,2,780,1140,1140,1140,25,1,12,1
1750-1775,1,1054,25,355,1762,45,53,6,33,1,...,1,2,1079,1762,1762,1762,25,1,12,1
1775-1800,1,1589,25,482,2810,47,62,7,41,1,...,1,2,1483,2810,2810,2810,25,1,12,1
1800-1825,1,3044,25,679,5102,89,123,9,81,1,...,1,1,2333,5102,5102,5102,25,1,12,1
1825-1850,1,3925,25,884,6616,123,168,10,107,1,...,1,2,2889,6616,6616,6616,25,1,12,1
1850-1875,1,5390,25,1074,9311,122,138,9,103,1,...,1,2,3674,9311,9311,9311,25,1,12,1
1875-1900,1,7339,25,1245,12966,141,174,13,117,1,...,1,2,4058,12966,12966,12966,25,1,12,1


In [42]:
def get_dfinfo_grp(by='period'):
    dfinfo = dfmeta.join(
        dfocr,rsuffix='_ocr'
    ).groupby(by).sum().join(
        dfskip.groupby('period').sum(),
        lsuffix='_meta'
    )[['num_words_ocr','num_recog','count','num_lines','num_words']].join(
        orig_stats
    )
    #dfinfo['num_journals']=dfmeta.groupby('period').Publication_PublicationID.nunique()
    dfinfo.loc['(Total)']=dfinfo.sum()
    dfinfo.loc['(Total)', 'num_journals']=dfmeta.Publication_PublicationID.nunique()

    dfinfo['ocr_accuracy'] = [x/y for x,y in zip(
        dfinfo.num_recog,dfinfo.num_words_ocr
    )]
    return dfinfo

In [43]:
dfinfo=get_dfinfo_grp('period')
dfinfo

,num_words_ocr,num_recog,count,num_lines,num_words,num_journals,num_articles,ocr_accuracy
period,,,,,,,,
1675-1700,4.452787e+06,3.207739e+06,174.0,51395.0,1.284875e+06,21.0,3918.0,0.720389
1700-1725,1.337782e+07,9.966688e+06,595.0,191799.0,4.794975e+06,44.0,13661.0,0.745016
1725-1750,4.551386e+07,3.415612e+07,1140.0,598702.0,1.496755e+07,38.0,42427.0,0.750455
1750-1775,7.361105e+07,5.631703e+07,1762.0,1044973.0,2.612432e+07,45.0,70245.0,0.765062
1775-1800,1.580828e+08,1.171691e+08,2810.0,1951906.0,4.879765e+07,47.0,145531.0,0.741188
1800-1825,2.971293e+08,2.397860e+08,5102.0,3701248.0,9.253120e+07,89.0,202538.0,0.807009
1825-1850,4.814221e+08,4.214009e+08,6616.0,5701521.0,1.425380e+08,123.0,279701.0,0.875325
1850-1875,8.446339e+08,7.494083e+08,9311.0,8921359.0,2.230340e+08,122.0,445557.0,0.887258
1875-1900,1.010547e+09,9.125688e+08,12966.0,11234409.0,2.808602e+08,141.0,699344.0,0.903044


In [44]:
odf = dfinfo[[
    'num_journals','num_articles','num_lines','num_words',
    'num_words_ocr'
]].applymap(lambda x: f'{int(x):,}').rename_axis('Period')
odf['ocr_accuracy']=dfinfo.ocr_accuracy.apply(lambda x: f'{(x*100):.0f}%')
odf=odf[['num_journals','num_articles','num_words_ocr','ocr_accuracy','num_lines','num_words']]
odf.columns=['# Journals','# Articles','# Words','%OCR acc.', '# Sents (model)', '# Words (model)']
odf

,# Journals,# Articles,# Words,%OCR acc.,# Sents (model),# Words (model)
Period,,,,,,
1675-1700,21,"3,918","4,452,787",72%,"51,395","1,284,875"
1700-1725,44,"13,661","13,377,824",75%,"191,799","4,794,975"
1725-1750,38,"42,427","45,513,862",75%,"598,702","14,967,550"
1750-1775,45,"70,245","73,611,046",77%,"1,044,973","26,124,325"
1775-1800,47,"145,531","158,082,845",74%,"1,951,906","48,797,650"
1800-1825,89,"202,538","297,129,281",81%,"3,701,248","92,531,200"
1825-1850,123,"279,701","481,422,103",88%,"5,701,521","142,538,025"
1850-1875,122,"445,557","844,633,887",89%,"8,921,359","223,033,975"
1875-1900,141,"699,344","1,010,547,113",90%,"11,234,409","280,860,225"


In [47]:
import dataframe_image as dfi

In [48]:
df_to_png(odf,os.path.join(PATH_FIGS2,'fig.corpus1-v6.png'))

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\x00\x00\x00\x01\x9a\x08\x06\x00\x00\x00\x91\xd81…

'/home/ryan/Markdown/Drafts/TheGreatAbstraction/figures/fig.corpus1-v6.png'

In [46]:
stop

NameError: name 'stop' is not defined

In [ ]:
dfi.export(odf, os.path.join(PATH_FIGS2,'fig.corpus1-v5.png'))

In [ ]:
meta=dfmeta.join(dfocr,rsuffix='_ocr')
# meta['Period']=meta.year.apply(lambda y: periodize_bystep(y,20,1700,1900))
# meta=meta[meta.Period!='']
meta['Period']=meta['period']
mdf=meta.groupby(['Period','Genre']).num_words_ocr.sum().reset_index()
mpiv=mdf.pivot('Genre','Period','num_words_ocr').fillna(0)
mpiv=mpiv[mpiv.columns[1:-1]]
for col in mpiv:
    mpiv[col]=mpiv[col]/mpiv[col].sum()
mpiv['sum']=mpiv.sum(axis=1)
mpiv=mpiv.sort_values('sum',ascending=False).drop('sum',1).rename_axis('Genre')
mpiv=mpiv.style.background_gradient('gray_r',vmin=0,vmax=1)
mpiv=mpiv.format('{:.2%}')
mpiv

In [ ]:
dfi.export(mpiv, os.path.join(PATH_FIGS2,'fig.corpus3-v7.png'))

In [ ]:
# dfmeta.style.background_gradient??

In [ ]:
dfmeta.num_words=pd.to_numeric(dfmeta.num_words)

In [ ]:
words=dfmeta.groupby('Genre').agg(['sum','mean']).drop('year',1)
words.columns=['_'.join(x) for x in words.columns.to_flat_index()]
words['perc']=words['num_words_sum'] / words['num_words_sum'].sum() * 100
words=words.sort_values('perc',ascending=False)
words=words.applymap(lambda x: f'{x:,.0f}')
words['perc']=words['perc'].apply(lambda x: f'{x}%')
words=words[['num_words_sum','perc','num_words_mean']]
words.columns=['# Words','% Words', '# Words (avg.)']
# print(f'\n\n{words.to_markdown()}\n\n')

In [ ]:
dfinfogenre = genres.join(words).sort_values('count',ascending=False).drop('count',1)
dfinfogenre

In [ ]:
dfi.export(dfinfogenre, os.path.join(PATH_FIGS2,'fig.corpus2.png'))

In [ ]:
print(dfinfogenre.to_markdown())

In [ ]:
dfmeta[dfmeta.ObjectType=='General Information'].index[0]

In [ ]:
t=C.textd['1642/General_Information/2032695']
t

In [ ]:
C.meta.num_words=C.meta.num_words.apply(float)
C.meta.num_words.mean()

In [ ]:
C.meta.groupby('ObjectType').sum()

In [ ]:
dfgpinfo=dfmeta.groupby(['period','Genre']).sum().reset_index()
dfgpinfo['year']=[int(y[:4]) for y in dfgpinfo.period]
dfgpinfo=pd.concat(grp.assign(perc_words=grp.num_words / grp.num_words.sum()) for i,grp in dfgpinfo.groupby('period'))
dfgpinfo

In [ ]:
dfgpinfo_mean=dfgpinfo.groupby('Genre').mean().sort_values('perc_words',ascending=False)
dfgpinfo_mean

In [ ]:
dfgpinfo['Genre']=pd.Categorical(dfgpinfo['Genre'], categories=dfgpinfo_mean.index)
dfgpinfo

In [ ]:
fig=p9.ggplot(
    p9.aes(x='period',y='perc_words',group='Genre'),
    data=dfgpinfo.query('1720<=year<1900')
)
fig+=p9.theme_minimal()
fig+=p9.theme(text=p9.element_text(size=6), axis_text_x=p9.element_text(rotation=90,size=6))
fig+=p9.geom_line()
fig+=p9.facet_wrap('Genre')
p9.options.figure_size=6,6
fig